In [1]:
import os
import re
import random
import pprint
import mne
from mne.io.edf.edf import RawEDF
import numpy as np
from datetime import datetime
import pandas as pd
import pywt
from scipy.stats import skew
import math


# Feature Extraction

The most discriminative features for CHB-MIT dataset (Chen et. al, 2017):

- Energy
- Max
- Min
- Mean
- Standard Deviation
- Skewness

6 features for each sub-band resulting in 6 x 6 = 36 features for channel.

## Discrete Wavelet Transformation

Discrete Wavelet Transformation (DWT) is a transform that decomposes a given signal into a number of sets, 
where each set is a time series of coefficients describing the time evolution of the signal in the corresponding frequency band




In [2]:
def feature_vector(data, times):

    sampling_freq = data.info['sfreq']
    start_stop_seconds = np.array(times)
    window_size = start_stop_seconds[1]-start_stop_seconds[0]
    start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
    eeg_channel_indices = mne.pick_types(data.info, meg=False, eeg=True)
    eeg_data, times = data[eeg_channel_indices, start_sample:stop_sample]

    coeffs = pywt.wavedec(eeg_data[0:23], "coif3", level=7)
    cA7, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
    
    band1_mean,band2_mean,band3_mean,band4_mean,band5_mean,band6_mean = [],[],[],[],[],[] # mean
    band1_std,band2_std,band3_std,band4_std,band5_std,band6_std = [],[],[],[],[],[] # standard deviation
    band1_en, band2_en, band3_en, band4_en, band5_en, band6_en = [],[],[],[],[],[] # energy
    band1_max, band2_max, band3_max, band4_max, band5_max, band6_max = [],[],[],[],[],[] # max 
    band1_min, band2_min, band3_min, band4_min, band5_min, band6_min = [],[],[],[],[],[] # min
    band1_skew, band2_skew, band3_skew, band4_skew, band5_skew, band6_skew = [],[],[],[],[],[] #skewness

    # Calculate 6 features of DWT coefficients.
    for i in range(len(cD1)):
        band1_en.append(np.sum(cD7[i, :] ** 2))
        band2_en.append(np.sum(cD6[i, :] ** 2))
        band3_en.append(np.sum(cD5[i, :] ** 2))
        band4_en.append(np.sum(cD4[i, :] ** 2))
        band5_en.append(np.sum(cD3[i, :] ** 2))
        band6_en.append(np.sum(cD2[i, :] ** 2))

        band1_max.append(np.max(cD7[i, :]))
        band2_max.append(np.max(cD6[i, :]))
        band3_max.append(np.max(cD5[i, :]))
        band4_max.append(np.max(cD4[i, :]))
        band5_max.append(np.max(cD3[i, :]))
        band6_max.append(np.max(cD2[i, :]))

        band1_min.append(np.min(cD7[i, :]))
        band2_min.append(np.min(cD6[i, :]))
        band3_min.append(np.min(cD5[i, :]))
        band4_min.append(np.min(cD4[i, :]))
        band5_min.append(np.min(cD3[i, :]))
        band6_min.append(np.min(cD2[i, :]))

        band1_mean.append(np.mean(cD7[i, :]))
        band2_mean.append(np.mean(cD6[i, :]))
        band3_mean.append(np.mean(cD5[i, :]))
        band4_mean.append(np.mean(cD4[i, :]))
        band5_mean.append(np.mean(cD3[i, :]))
        band6_mean.append(np.mean(cD2[i, :]))

        band1_std.append(np.std(cD7[i, :]))
        band2_std.append(np.std(cD6[i, :]))
        band3_std.append(np.std(cD5[i, :]))
        band4_std.append(np.std(cD4[i, :]))
        band5_std.append(np.std(cD3[i, :]))
        band6_std.append(np.std(cD2[i, :]))

        band1_skew.append(skew(cD7[i, :]))
        band2_skew.append(skew(cD6[i, :]))
        band3_skew.append(skew(cD5[i, :]))
        band4_skew.append(skew(cD4[i, :]))
        band5_skew.append(skew(cD3[i, :]))
        band6_skew.append(skew(cD2[i, :]))

    band1_en = (np.array(band1_en).reshape(1, -1))
    band2_en = (np.array(band2_en).reshape(1, -1))
    band3_en = (np.array(band3_en).reshape(1, -1))
    band4_en = (np.array(band4_en).reshape(1, -1))
    band5_en = (np.array(band5_en).reshape(1, -1))
    band6_en = (np.array(band6_en).reshape(1, -1))

    band1_max = np.array(band1_max).reshape(1, -1)
    band2_max = np.array(band2_max).reshape(1, -1)
    band3_max = np.array(band3_max).reshape(1, -1)
    band4_max = np.array(band4_max).reshape(1, -1)
    band5_max = np.array(band5_max).reshape(1, -1)
    band6_max = np.array(band6_max).reshape(1, -1)

    band1_min = np.array(band1_min).reshape(1, -1)
    band2_min = np.array(band2_min).reshape(1, -1)
    band3_min = np.array(band3_min).reshape(1, -1)
    band4_min = np.array(band4_min).reshape(1, -1)
    band5_min = np.array(band5_min).reshape(1, -1)
    band6_min = np.array(band6_min).reshape(1, -1)

    band1_mean = np.array(band1_mean).reshape(1, -1)
    band2_mean = np.array(band2_mean).reshape(1, -1)
    band3_mean = np.array(band3_mean).reshape(1, -1)
    band4_mean = np.array(band4_mean).reshape(1, -1)
    band5_mean = np.array(band5_mean).reshape(1, -1)
    band6_mean = np.array(band6_mean).reshape(1, -1)

    band1_std = np.array(band1_std).reshape(1, -1)
    band2_std = np.array(band2_std).reshape(1, -1)
    band3_std = np.array(band3_std).reshape(1, -1)
    band4_std = np.array(band4_std).reshape(1, -1)
    band5_std = np.array(band5_std).reshape(1, -1)
    band6_std = np.array(band6_std).reshape(1, -1)

    band1_skew = np.array(band1_skew).reshape(1, -1)
    band2_skew = np.array(band2_skew).reshape(1, -1)
    band3_skew = np.array(band3_skew).reshape(1, -1)
    band4_skew = np.array(band4_skew).reshape(1, -1)
    band5_skew = np.array(band5_skew).reshape(1, -1)
    band6_skew = np.array(band6_skew).reshape(1, -1)

    feature_vector = np.concatenate((band1_en, band1_max, band1_min, band1_mean, band1_std, band1_skew,
                                    band2_en, band2_max, band2_min, band2_mean, band2_std, band2_skew,
                                    band3_en, band3_max, band3_min, band3_mean, band3_std, band3_skew,
                                    band4_en, band4_max, band4_min, band4_mean, band4_std, band4_skew,
                                    band5_en, band5_max, band5_min, band5_mean, band5_std, band5_skew,
                                    band6_en, band6_max, band6_min, band6_mean, band6_std, band6_skew
                                        ), axis=0)

    # feature vector if 23 x 36 
    return feature_vector 

# Epochs
Since EEG-signals are continuous and non-stationary, epochs are then defined as time segments; in our case, a 2-second epoch has the dimension of 23 x 512 (sample size 256 Hz)

## Create wavelet features for 3 sec epochs of ictal-states
We try and take the middle of the signal 

In [3]:
def feature_vectors_ictal(df, seconds):
    overall_features = []
    rootdir = f'{os.getcwd()}/data/'
    seconds = seconds/2
    
    for i in range(df.shape[0]):
        folder = df['case'][i]
        filename = df['seizure_file'][i]
        time = round((df['seizure_start'][i]+df['seizure_end'][i])/2)
        
        data = mne.io.read_raw_edf(input_fname=f'{rootdir}{folder}/{filename}.edf', preload=False, verbose='Error')
        times = [round(time - seconds), round(time + seconds)]
        overall_features.append(feature_vector(data, times)) 
            

    overall_features = np.reshape(overall_features, (np.shape(overall_features[0]),23*36))
    
    return overall_features

    #savename =  case + '_feature_ictal' + '.npy'
    #dir = os.path.join(rootdir+case+"/feature_ictal")
        
    #if not os.path.exists(dir):
    #    os.mkdir(dir)
                
    #savepath = f'{dir}/{savename}'
    #np.save(savepath, overall_features)

In [11]:
df = pd.read_excel('output.xlsx', index_col=0)
seconds = 3/2
rootdir = f'{os.getcwd()}/data/'

folder = df['case'][0]
filename = df['seizure_file'][0]
time = round((df['seizure_start'][0]+df['seizure_end'][0])/2)


times = [math.floor(time - seconds), math.floor(time + seconds)]
times

data = mne.io.read_raw_edf(input_fname=f'{rootdir}{folder}/{filename}.edf', preload=False, verbose='Error')
np.shape(feature_vector(data, times))

/Users/martinnguyen/opt/anaconda3/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


(36, 23)

In [15]:
df = pd.read_excel('output.xlsx', index_col=0)
seconds = 5

ictal_vector = feature_vectors_ictal(df, seconds)

/Users/martinnguyen/opt/anaconda3/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


ValueError: No data in this range

'chb01'